In [1]:
import subprocess
import time
import numpy as np
import pandas as pd
import os

# set a shell file
each line in this file is a job to trace, it could be any commomd line

# tmp.py file
it could be anything

# functions to trace memory and time

In [2]:
def mem_of_one_line(lines, sleep_time): # lines: a list
    proc = subprocess.Popen(lines)
    pid = proc.pid
    info_dict = get_mem_of_a_pid(pid, sleep_time)
    return info_dict['VmHWM:'], info_dict['VmRSS:'] # peak RAM, current RAM
    
def str_in_kb_to_float_in_byte(text):
    if not 'kB' in text:
        print('please check if the VmHWM and VmRSS in /proc/PID/status file are in kB')
    return float(text.replace('kB', ''))*1024

def get_info(status_file):
    f = open(status_file, 'rt')
    need_to_be_extracted = ['State:', 'VmHWM:', 'VmRSS:']
    info_dict = {}
    for line in f:
        lines = line.strip().split('\t')
        if np.isin(lines[0], need_to_be_extracted):
            info_dict[lines[0]] = lines[1]
    if 'VmHWM:' in info_dict and 'VmRSS:' in info_dict:
        info_dict['VmHWM:'] = str_in_kb_to_float_in_byte(info_dict['VmHWM:'])
        info_dict['VmRSS:'] = str_in_kb_to_float_in_byte(info_dict['VmRSS:'])

    return info_dict

def get_mem_of_a_pid(pid, sleep_time):
    status_file = f'/proc/{pid}/status'
    info_dict_previous = get_info(status_file)
    # print(pid)
    while(True):
        info_dict_now = get_info(status_file)
        if 'Z' in info_dict_now['State:'] or not os.path.exists(status_file):
            # print(info_dict_now)
            return info_dict_previous
        else:
            info_dict_previous = info_dict_now
            time.sleep(sleep_time)

def get_mem_and_time_of_a_shell(shell_file, time_interval_of_two_traces=10):
    # mem in byte, time in second
    f = open(shell_file, 'rt')
    peak_rams, curr_rams, times = [], [], []
    for line in f:
        lines = line.strip().split()
        s = time.time()
        peak, curr = mem_of_one_line(lines, time_interval_of_two_traces)
        e = time.time()
        peak_rams.append(peak)
        curr_rams.append(curr)
        times.append(e-s)
    return pd.DataFrame({'peak_size': peak_rams, 'current_size': curr_rams, 'time': times})

# how to use

In [3]:
mem_time_used_df = get_mem_and_time_of_a_shell('test.sh', time_interval_of_two_traces=2)
mem_time_used_df

1
[1, 2, 3]
finish
2
[1, 2, 3]
finish


,peak_size,current_size,time
0,415256576.0,415256576.0,12.045828
1,415248384.0,415248384.0,12.050008
